In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [2]:
## Extracting features of 2017 movies from Wikipedia

In [3]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2017"

In [4]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [5]:
tables = soup.find_all('table',class_='wikitable sortable')

In [6]:
len(tables)

4

In [7]:
type(tables[0])

bs4.element.Tag

In [8]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [9]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_15460\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [10]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,J A N U A R Y,6,Underworld: Blood Wars,Screen Gems / Lakeshore Entertainment / Sketch...,Anna Foerster (director); Cory Goodman (screen...,[2][3],NaN
1,J A N U A R Y,6,Arsenal,Lionsgate Premiere,Steven C. Miller (director); Jason Mosberg (sc...,[4][5],NaN
2,J A N U A R Y,6,Between Us,IFC Films / Nora Films,Rafael Palacio Illingworth (director/screenpla...,[6],NaN
3,J A N U A R Y,13,Monster Trucks,Paramount Pictures / Paramount Animation / Nic...,Chris Wedge (director); Derek Connolly (screen...,[7][8],NaN
4,J A N U A R Y,13,The Bye Bye Man,STX Entertainment / Intrepid Pictures,Stacy Title (director); Jonathan Penner (scree...,[9],NaN
...,...,...,...,...,...,...,...
242,D E C E M B E R,22,Crooked House,Vertical Entertainment / Sony Pictures,Gilles Paquet-Brenner (director/screenplay); T...,NaN,[293]
243,D E C E M B E R,22,Hostiles,Entertainment Studios,Scott Cooper (director/screenplay); Christian ...,NaN,[294]
244,D E C E M B E R,25,All the Money in the World,TriStar Pictures,Ridley Scott (director); David Scarpa (screenp...,NaN,[295]
245,D E C E M B E R,25,Molly's Game,STX Entertainment,Aaron Sorkin (director/screenplay); Jessica Ch...,NaN,[296]


In [11]:
df_2017 = df[['Title','Cast and crew']]

In [12]:
df_2017

,Title,Cast and crew
0,Underworld: Blood Wars,Anna Foerster (director); Cory Goodman (screen...
1,Arsenal,Steven C. Miller (director); Jason Mosberg (sc...
2,Between Us,Rafael Palacio Illingworth (director/screenpla...
3,Monster Trucks,Chris Wedge (director); Derek Connolly (screen...
4,The Bye Bye Man,Stacy Title (director); Jonathan Penner (scree...
...,...,...
242,Crooked House,Gilles Paquet-Brenner (director/screenplay); T...
243,Hostiles,Scott Cooper (director/screenplay); Christian ...
244,All the Money in the World,Ridley Scott (director); David Scarpa (screenp...
245,Molly's Game,Aaron Sorkin (director/screenplay); Jessica Ch...


In [13]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'ff3132c29c92e21e14539982f56e664a'

In [14]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [ ]:
df_2017['genres'] = df_2017['Title'].map(lambda x: get_genre(str(x)))

In [ ]:
df_2017

In [ ]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [ ]:
df_2023['director_name'] = df_2023['Cast and crew'].map(lambda x: get_director(str(x)))

In [ ]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [ ]:
df_2017['actor_1_name'] = df_2017['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [ ]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [ ]:
df_2017['actor_2_name'] = df_2017['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [ ]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [ ]:
df_2017['actor_3_name'] = df_2017['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [ ]:
df_2017

In [ ]:
df_2017 = df_2017.rename(columns={'Title':'movie_title'})

In [ ]:
new_df17 = df_2017.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
new_df17

In [ ]:
new_df17['comb'] = new_df17['actor_1_name'] + ' ' + new_df17['actor_2_name'] + ' '+ new_df17['actor_3_name'] + ' '+ new_df17['director_name'] +' ' + new_df17['genres']

In [ ]:
new_df17.isna().sum()

In [ ]:
new_df17 = new_df17.dropna(how='any')

In [ ]:
new_df17.isna().sum()

In [ ]:
new_df17['movie_title'] = new_df17['movie_title'].str.lower()

In [ ]:
new_df17

In [ ]:
old_df = pd.read_csv('datasets/final_data.csv')

In [ ]:
old_df

In [ ]:
final_df = old_df.append(new_df23,ignore_index=True)

In [ ]:
final_df

In [ ]:
final_df.to_csv('main_data.csv',index=False)